<br>

<div align=center><font color=maroon size=6><b>Extending PyTorch</b></font></div>

<font size=4><b>References:</b></font>
* <a href="https://pytorch.org/docs/stable/index.html" style="text-decoration:none;">Docs > PyTorch documentation</a>

    * **Notes**
        * Docs > 7 <a href="https://pytorch.org/docs/stable/notes/extending.html" style="text-decoration:none;">Extending PyTorch</a>


<br>
<br>
<br>

<font size=3>In this note we’ll cover ways of extending `torch.nn`, <a href="https://pytorch.org/docs/stable/autograd.html#module-torch.autograd" style="text-decoration:none;"><font color=maroon>torch.autograd</font></a>, `torch`, and writing `custom C extensions` utilizing our C libraries.</font>

<br>
<br>
<br>

# Extending <font color=maroon><b>torch.autograd</b></font>

Adding operations to `torch.autograd` requires implementing a new <a href="https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function" style="text-decoration:none;"><font color=maroon size=3>Function</font></a> subclass for each operation. <br>
Recall that Functions are what `torch.autograd` uses to encode the operation history and compute gradients.

The first part of this doc is focused on <font size=3 color=maroon><b>backward mode AD</b> as it is the most widely used feature.</font><br>
A section at the end discusses the extensions for <font size=3 color=maroon><b>forward mode AD</b></font>.

<br>

## When to use

In general, implement a custom function if you want to perform computations in your model that are <font color=maroon size=3><b>not differentiable</b></font> or <font color=maroon size=3><b>rely on non-Pytorch libraries</b> (e.g., NumPy), but still wish for your operation to chain with other ops and work with the autograd engine.</font>

In some situations, custom functions can also be used to improve performance and memory usage: If you implemented your forward and backward passes using a <a href="https://pytorch.org/tutorials/advanced/cpp_extension.html" style="text-decoration:none;"><font color=maroon>C++ extension</font></a> `Tutorials > Custom C++ and CUDA Extensions`, you can wrap them in `Function` to interface with the autograd engine. If you’d like to reduce the number of buffers saved for the backward pass, custom functions can be used to combine ops together.

<br>
<br>

## When not to use

If you can already write your function in terms of PyTorch’s built-in ops, its backward graph is (most likely) already able to be recorded by autograd. In this case, you do not need to implement the backward function yourself. Consider using a plain old Python function.

If you need to maintain state, i.e., trainable parameters, you should (also) use a custom module. See the section below for more information on extending `torch.nn`.

If you’d like to alter the gradients during the backward pass or perform a side effect, consider registering a <a href="https://pytorch.org/docs/stable/generated/torch.Tensor.register_hook.html#torch.Tensor.register_hook" style="text-decoration:none;"><font color=maroon></font></a> or <a href="https://pytorch.org/docs/stable/notes/modules.html#module-hooks" style="text-decoration:none;"><font color=maroon>tensor
Module</font></a> hook.

<br>
<br>

## How to use

Take the following steps: 
1. Subclass <a href="https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function" style="text-decoration:none;"><font color=maroon><b>Function</b></font></a> and implement the <a href="https://pytorch.org/docs/stable/generated/torch.autograd.Function.forward.html#torch.autograd.Function.forward" style="text-decoration:none;"><font color=maroon>forward()</font></a> and <a href="https://pytorch.org/docs/stable/generated/torch.autograd.Function.backward.html#torch.autograd.Function.backward" style="text-decoration:none;"><font color=maroon>backward()</font></a> methods. 
2. Call the proper methods on the ***ctx*** argument. 
3. Declare whether your function supports double backward. 
4. Validate whether your gradients are correct using gradcheck.

<br>

<font size=3><b>Step 1:</b> After subclassing Function, you’ll need to define 2 methods:</font>

* <font color=maroon size=3><b>forward()</b></font> is the code that performs the operation. It can take as many arguments as you want, with some of them being optional, if you specify the default values. All kinds of Python objects are accepted here.<br>
`Tensor` arguments that track history (i.e., with `requires_grad=True`) will be converted to ones that don’t track history before the call, and their use will be registered in the graph. <br>
<font color=maroon>Note that</font> this logic won’t traverse lists/dicts/any other data structures and will only consider tensors that are direct arguments to the call. You can return either a single `Tensor` output, or a `tuple` of tensors if there are multiple outputs. Also, please refer to the docs of `Function` to find descriptions of useful methods that can be called only from `forward()`.

<br>

* <font color=maroon size=3><b>backward()</b></font> (or <font color=maroon size=3><b>vjp()</b></font>) defines the gradient formula. It will be given as many `Tensor` arguments as there were outputs, with each of them representing gradient w.r.t. that output. <font color=maroon>It is important NEVER to modify these in-place.</font> It should return as many tensors as there were inputs, with each of them containing the gradient w.r.t. its corresponding input. <br>
If your inputs didn’t require gradient (`needs_input_grad` is a tuple of booleans indicating whether each input needs gradient computation), or were non-Tensor objects, you can return `None`. Also, if you have optional arguments to `forward()` you can return more gradients than there were inputs, as long as they’re all `None`.

<br>

<font size=3><b>Step 2:</b> It is your responsibility to use the functions in the forward’s ***ctx*** properly in order to ensure that the new `Function` works properly with the `autograd engine`.</font>

* <a href="https://pytorch.org/docs/stable/generated/torch.autograd.function.FunctionCtx.save_for_backward.html#torch.autograd.function.FunctionCtx.save_for_backward" style="text-decoration:none;"><font color=maroon size=3>save_for_backward()</font></a> must be used when saving input or output tensors of the forward to be used later in the backward. Anything else, i.e., non-tensors and tensors that are neither input nor output should be stored directly on ***ctx***.


* <a href="https://pytorch.org/docs/stable/generated/torch.autograd.function.FunctionCtx.mark_dirty.html#torch.autograd.function.FunctionCtx.mark_dirty" style="text-decoration:none;"><font color=maroon size=3>mark_dirty()</font></a> must be used to mark any input that is modified inplace by the forward function.


* <a href="https://pytorch.org/docs/stable/generated/torch.autograd.function.FunctionCtx.mark_non_differentiable.html#torch.autograd.function.FunctionCtx.mark_non_differentiable" style="text-decoration:none;"><font color=maroon size=3>mark_non_differentiable()</font></a> must be used to tell the engine if an output is not differentiable. By default all output tensors that are of differentiable type will be set to require gradient. Tensors of non-differentiable type (i.e., integral types) are never marked as requiring gradients.


* <a href="https://pytorch.org/docs/stable/generated/torch.autograd.function.FunctionCtx.set_materialize_grads.html#torch.autograd.function.FunctionCtx.set_materialize_grads" style="text-decoration:none;"><font color=maroon size=3>set_materialize_grads()</font></a> can be used to tell the autograd engine to optimize gradient computations in the cases where the output does not depend on the input by not materializing grad tensors given to backward function. <br>
That is, if set to ``False``, ``None`` object in python or “undefined tensor” (tensor ``x`` for which ``x.defined()`` is ``False``) in C++ will not be converted to a tensor filled with zeros prior to calling backward, and so your code will need to handle such objects as if they were tensors filled with zeros. The default value of this setting is `True`.

<br>

<font size=3><b>Step 3:</b></font>

If your `Function` does not support double backward you should explicitly declare this by decorating backward with the <font color=maroon size=3>once_differentiable()</font>. With this decorator, attempts to perform double backward through your function will produce an error. See our double backward tutorial for more information on double backward.

<br>

<font size=3><b>Step 4:</b></font>

It is recommended that you use <a href="https://pytorch.org/docs/stable/generated/torch.autograd.gradcheck.html#torch.autograd.gradcheck" style="text-decoration:none;"><font color=maroon>torch.autograd.gradcheck()</font></a> to check whether your backward function correctly computes gradients of the forward by computing the Jacobian matrix using your backward function and comparing the value element-wise with the Jacobian computed numerically using finite-differencing.

<br>

## <font style="color:magenta;font-size:110%;font-weight:bold">Example</font>

Below you can find code for a `Linear` function from `torch.nn`, with additional comments:
```python
# Inherit from Function
class LinearFunction(Function):

    # Note that both forward and backward are @staticmethods
    @staticmethod
    # bias is an optional argument
    def forward(ctx, input, weight, bias=None):
        ctx.save_for_backward(input, weight, bias)
        output = input.mm(weight.t())
        if bias is not None:
            output += bias.unsqueeze(0).expand_as(output)
        return output

    # This function has only a single output, so it gets only one gradient
    @staticmethod
    def backward(ctx, grad_output):
        # This is a pattern that is very convenient - at the top of backward
        # unpack saved_tensors and initialize all gradients w.r.t. inputs to None. 
        # Thanks to the fact that additional trailing Nones are ignored, 
        # the return statement is simple even when the function has optional inputs.
        input, weight, bias = ctx.saved_tensors
        grad_input = grad_weight = grad_bias = None

        # These needs_input_grad checks are optional and there only to improve efficiency. 
        # If you want to make your code simpler, you can skip them. 
        # Returning gradients for inputs that don't require it is not an error.
        if ctx.needs_input_grad[0]:
            grad_input = grad_output.mm(weight)
        if ctx.needs_input_grad[1]:
            grad_weight = grad_output.t().mm(input)
        if bias is not None and ctx.needs_input_grad[2]:
            grad_bias = grad_output.sum(0)

        return grad_input, grad_weight, grad_bias
```

Now, to make it easier to use these custom ops, we recommend aliasing their apply method:
```python
linear = LinearFunction.apply
```

<br>

Here, we give an additional example of a function that is parametrized by non-Tensor arguments:
```python
class MulConstant(Function):
    @staticmethod
    def forward(ctx, tensor, constant):
        # ctx is a context object that can be used to stash information
        # for backward computation
        ctx.constant = constant
        return tensor * constant

    @staticmethod
    def backward(ctx, grad_output):
        # We return as many input gradients as there were arguments.
        # Gradients of non-Tensor arguments to forward must be None.
        return grad_output * ctx.constant, None
```

<br>

And here, we optimize the above example by calling `set_materialize_grads(False)`:
```python
class MulConstant(Function):
    @staticmethod
    def forward(ctx, tensor, constant):
        ctx.set_materialize_grads(False)
        ctx.constant = constant
        return tensor * constant

    @staticmethod
    def backward(ctx, grad_output):
        # Here we must handle None grad_output tensor. In this case we
        # can skip unnecessary computations and just return None.
        if grad_output is None:
            return None, None

        # We return as many input gradients as there were arguments.
        # Gradients of non-Tensor arguments to forward must be None.
        return grad_output * ctx.constant, None
```

<div class="alert alert-block alert-info">

<font size=3 color=red><b>NOTE: </b></font>

Inputs to `backward`, i.e., `grad_output`, can also be tensors that track history. So if `backward` is implemented with differentiable operations, (e.g., invocation of another custom `Function`), higher order derivatives will work. In this case, the tensors saved with `save_for_backward` can also be used in the backward and have gradients flowing back but tensors saved in the `ctx` won’t have gradients flowing back for them. If you need gradients to flow back for a Tensor saved in the `ctx`, you should make it an output of the custom `Function` and save it with `save_for_backward`.

</div>

<br>

You probably want to check if the backward method you implemented actually computes the derivatives of your function. It is possible by comparing with numerical approximations using small finite differences:
```python
from torch.autograd import gradcheck

# gradcheck takes a tuple of tensors as input, check if your gradient
# evaluated with these tensors are close enough to numerical
# approximations and returns True if they all verify this condition.
input = (torch.randn(20,20,dtype=torch.double,requires_grad=True),
         torch.randn(30,20,dtype=torch.double,requires_grad=True))
test = gradcheck(linear, input, eps=1e-6, atol=1e-4)
print(test)
```

See `Docs > Automatic differentiation package - torch.autograd > `<a href="https://pytorch.org/docs/stable/autograd.html#grad-check" style="text-decoration:none;"><font color=maroon>Numerical gradient checking</font></a> for more details on finite-difference gradient comparisons. If your function is used in higher order derivatives (differentiating the backward pass) you can use the `gradgradcheck` function from the same package to check higher order derivatives.

<br>
<br>

## Forward mode AD

Overriding the forward mode AD formula has a very similar API with some different subtleties. You can implement the <a href="https://pytorch.org/docs/stable/generated/torch.autograd.Function.jvp.html#torch.autograd.Function.jvp" style="text-decoration:none;"><font color=maroon size=4><b>jvp()</b></font></a> function.

It will be given as many Tensor arguments as there were inputs, with each of them representing gradient w.r.t. that input. It should return as many tensors as there were outputs, with each of them containing the gradient w.r.t. its corresponding output. The `jvp()` will be called just after the `forward()` method, before the `apply()` returns.

<font size=3 color=maroon><b>jvp()</b> has a few subtle differences with the <b>backward()</b> function:</font>

* You can use the ctx to pass any data from the `forward()` to the `jvp()` function. If that state will not be needed for the `backward()`, you can explicitly free it by doing del ctx.foo at the end of the `jvp()` function.


* The implementation of `jvp()` must be backward differentiable or explicitly check that none of the given forward mode gradient has **`requires_grad set`**.


* The `jvp()` function must match the view/inplace behavior of `forward()`. For example, if the `i` th input is modified inplace, then the `i` th gradient must be updated inplace. Similarly, if the `j` th output is a view of the `k` th input. Then the returned `j` th output gradient must be a view of the given `k` th input gradient.


* Because the user cannot specify which gradient needs to be computed, the `jvp()` function should always compute gradients for all the outputs.


* The forward mode gradients do respect the flag set by **`set_materialize_grads()`** and you can get None input gradients when this is disabled.

<br>
<br>
<br>

# Extending <font color=maroon><b>torch.nn</b></font>

<font size=3><font color=maroon>`nn` exports two kinds of interfaces - <b>modules</b> and <b>their functional versions</b>.</font> You can extend it in both ways, 
* but we recommend using `modules` for all kinds of layers, that hold any parameters or buffers, 


* and recommend using a `functional` for~m~ parameter-less operations like activation functions, pooling, etc.</font>

<br>

## Adding a functional version

<font size=4>Adding a functional version of an operation is already fully covered in the section above.</font>

<br>

## Adding a <a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module" style="text-decoration:none;">Module</a>

Since `nn` heavily utilizes `autograd`, adding a new `Module` requires implementing a `Function` that performs the operation and can compute the gradient. <br>
From now on let’s assume that we want to implement a `Linear module` and we have the function implemented as in the listing above. There’s very little code required to add this. Now, there are two functions that need to be implemented:
* **`__init__`** (optional) - takes in arguments such as kernel sizes, numbers of features, etc. and initializes parameters and buffers.


* **`forward()`** - instantiates a `Function` and uses it to perform the operation. It’s very similar to a functional wrapper shown above.

This is how a `Linear module` can be implemented:
```python
class Linear(nn.Module):
    def __init__(self, input_features, output_features, bias=True):
        super(Linear, self).__init__()
        self.input_features = input_features
        self.output_features = output_features

        # nn.Parameter is a special kind of Tensor, that will get automatically registered as 
        # Module's parameter once it's assigned as an attribute. 
        # Parameters and buffers need to be registered, or
        # they won't appear in .parameters() (doesn't apply to buffers), 
        # and won't be converted when e.g. .cuda() is called. 
        # You can use .register_buffer() to register buffers.
        #
        # nn.Parameters require gradients by default.
        self.weight = nn.Parameter(torch.empty(output_features, input_features))
        if bias:
            self.bias = nn.Parameter(torch.empty(output_features))
        else:
            # You should always register all possible parameters, but the
            # optional ones can be None if you want.
            self.register_parameter('bias', None)

        # Not a very smart way to initialize weights
        nn.init.uniform_(self.weight, -0.1, 0.1)
        if self.bias is not None:
            nn.init.uniform_(self.bias, -0.1, 0.1)

    def forward(self, input):
        # See the autograd section for explanation of what happens here.
        return LinearFunction.apply(input, self.weight, self.bias)

    def extra_repr(self):
        # (Optional)Set the extra information about this module. 
        # You can test it by printing an object of this class.
        return 'input_features={}, output_features={}, bias={}'.format(self.input_features, 
                                                                       self.output_features, 
                                                                       self.bias is not None )
```

<br>
<br>
<br>


# Extending <font color=maroon><b>torch</b></font>

You can create custom types that emulate `Tensor` by defining a custom class with methods that match `Tensor`. But what if you want to be able to pass these types to functions like 
<a href="https://pytorch.org/docs/stable/generated/torch.add.html#torch.add" style="text-decoration:none;">torch.add()</a> in the top-level `torch` namespace that accept `Tensor` operands?

<br>

If your custom python type defines a method named ***`__torch_function__`***, PyTorch will invoke your ***`__torch_function__`*** implementation when an instance of your custom class is passed to a function in the `torch` namespace.<br>

This makes it possible to define custom implementations for any of the functions in the `torch` namespace which your ***`__torch_function__`*** implementation can call, allowing your users to make use of your custom type with existing PyTorch workflows that they have already written for `Tensor`. <br>

This works with “duck” types that are unrelated to `Tensor` as well as user-defined subclasses of `Tensor`.

<br>

## Extending <font color=maroon>torch</font> with a <font color=maroon><b>Tensor-like</b></font> type

<div class="alert alert-block alert-info">

<font size=3 color=red><b>NOTE: </b></font>

This functionality is inspired by the NumPy `__array_function__` protocol. See <a href="https://docs.scipy.org/doc/numpy/user/basics.dispatch.html#basics-dispatch" style="text-decoration:none;"><font color=red>the NumPy documentation</font></a> and <a href="https://numpy.org/neps/nep-0018-array-function-protocol.html" style="text-decoration:none;"><font color=red>NEP-0018</font></a> for more details.

</div>

To make this concrete, let’s begin with a simple example that illustrates the API dispatch mechanism. We’ll create a custom type that represents a 2D scalar tensor, parametrized by the order `N` and value along the diagonal entries, `value`:

In [1]:
import torch

In [2]:
class ScalarTensor(object):
    
    def __init__(self, N, value):
        self._N = N
        self._value = value

    def __repr__(self):
        return "DiagonalTensor(N={}, value={})".format(self._N, self._value)

    def tensor(self):
        return self._value * torch.eye(self._N)

This first iteration of the design isn’t very useful. The main functionality of `ScalarTensor` is to provide a more compact string representation of a scalar tensor than in the base tensor class:

In [3]:
d = ScalarTensor(5, 2)
d

DiagonalTensor(N=5, value=2)

In [4]:
d.tensor()

tensor([[2., 0., 0., 0., 0.],
        [0., 2., 0., 0., 0.],
        [0., 0., 2., 0., 0.],
        [0., 0., 0., 2., 0.],
        [0., 0., 0., 0., 2.]])

<br>

If we try to use this object with the `torch` API, we will run into issues:

In [5]:
torch.mean(d)

TypeError: mean(): argument 'input' (position 1) must be Tensor, not ScalarTensor

<br>

Adding a `__torch_function__` implementation to `ScalarTensor` makes it possible for the above operation to succeed. Let’s re-do our implementation, this time adding a `__torch_function__` implementation:

In [6]:
HANDLED_FUNCTIONS = {}
class ScalarTensor(object):
    def __init__(self, N, value):
        self._N = N
        self._value = value

    def __repr__(self):
        return "DiagonalTensor(N={}, value={})".format(self._N, self._value)

    def tensor(self):
        return self._value * torch.eye(self._N)

    @classmethod
    def __torch_function__(cls, func, types, args=(), kwargs=None):
        if kwargs is None:
            kwargs = {}
        if func not in HANDLED_FUNCTIONS or not all(
            issubclass(t, (torch.Tensor, ScalarTensor)) for t in types):
            return NotImplemented
        return HANDLED_FUNCTIONS[func](*args, **kwargs)

The `__torch_function__` method takes four arguments: 
* `func`, a reference to the torch API function that is being overridden, 
* `types`, the list of types of Tensor-likes that implement `__torch_function__`, 
* `args`, the tuple of arguments passed to the function, 
* and `kwargs`, the dict of keyword arguments passed to the function. 

It uses a global dispatch table named `HANDLED_FUNCTIONS` to store custom implementations. The keys of this dictionary are functions in the `torch` namespace and the values are implementations for `ScalarTensor`.

<div class="alert alert-block alert-info">

<font size=3 color=red><b>NOTE: </b></font>

Using a global dispatch table is not a mandated part of the `__torch_function__` API, it is just a useful design pattern for structuring your override implementations.

</div>

<br>

This class definition isn’t quite enough to make torch.mean do the right thing when we pass it a `ScalarTensor` – we also need to define an implementation for `torch.mean` for `ScalarTensor` operands and add the implementation to the `HANDLED_FUNCTIONS` dispatch table dictionary. <font color=maroon>One way of doing this is to define a <b>decorator</b>:</font>

In [7]:
import functools
def implements(torch_function):
    """Register a torch function override for ScalarTensor"""
    @functools.wraps(torch_function)
    def decorator(func):
        HANDLED_FUNCTIONS[torch_function] = func
        return func
    return decorator

which can be applied to the implementation of our override:

In [8]:
@implements(torch.mean)
def mean(input):
    return float(input._value) / input._N

With this change we can now use `torch.mean` with `ScalarTensor`:

In [9]:
d = ScalarTensor(5, 2)
torch.mean(d)

0.4

<br>

Of course `torch.mean` is an example of the simplest kind of function to override since it only takes one operand. We can use the same machinery to override a function that takes more than one operand, any one of which might be a tensor or tensor-like that defines `__torch_function__`, for example for <a href="https://pytorch.org/docs/stable/generated/torch.add.html#torch.add" style="text-decoration:none;">torch.add()</a>:

In [10]:
def ensure_tensor(data):
    if isinstance(data, ScalarTensor):
        return data.tensor()
    return torch.as_tensor(data)


@implements(torch.add)
def add(input, other):
   try:
       if input._N == other._N:
           return ScalarTensor(input._N, input._value + other._value)
       else:
           raise ValueError("Shape mismatch!")
   except AttributeError:
       return torch.add(ensure_tensor(input), ensure_tensor(other))

This version has a fast path for when both operands are `ScalarTensor` instances and also a slower path which degrades to converting the data to tensors when either operand is not a `ScalarTensor`. That makes the override function correctly when either operand is a `ScalarTensor` or a regular `Tensor`:

In [11]:
s = ScalarTensor(2, 2)
torch.add(s, s)

DiagonalTensor(N=2, value=4)

In [12]:
t = torch.tensor([[1, 1,], [1, 1]])
torch.add(s, t)

tensor([[3., 1.],
        [1., 3.]])

<br>

Note that our implementation of `add` does not take `alpha` or `out` as keyword arguments like <a href="https://pytorch.org/docs/stable/generated/torch.add.html#torch.add" style="text-decoration:none;">torch.add()</a> does:

In [13]:
torch.add(s, s, alpha=2)

TypeError: add() got an unexpected keyword argument 'alpha'

For speed and flexibility the `__torch_function__` dispatch mechanism does not check that the signature of an override function matches the signature of the function being overrided in the `torch` API. For some applications ignoring optional arguments would be fine but to ensure full compatibility with `Tensor`, user implementations of torch API functions should take care to exactly emulate the API of the function that is being overrided.

Functions in the torch API that do not have explicit overrides will return `NotImplemented` from `__torch_function__`. If all operands with `__torch_function__` defined on them return `NotImplemented`, PyTorch will raise a `TypeError`. This means that most of the time operations that do not have explicit overrides for a type will raise a `TypeError` when an instance of such a type is passed:

In [14]:
torch.mul(s, 3)

TypeError: no implementation found for 'torch.mul' on types that implement __torch_function__: [000001B58B30C990]

原文档中的：
```python
TypeError: no implementation found for 'torch.mul' on types that implement __torch_function__: [ScalarTensor]
```

<br>

<font size=3>In practice this means that if you would like to implement your overrides using a `__torch_function__` implementation along these lines, you will need to explicitly implement the full `torch` API or the entire subset of the API that you care about for your use case. <font color=maroon>This may be a tall order as the full `torch` API is quite extensive.</font></font>

<font size=3 color=maroon>Another option is to not return `NotImplemented` for operations that are not handled but to instead pass a `Tensor` to the original `torch` function when no override is available.</font> For example, if we change our implementation of `__torch_function__` for `ScalarTensor` to the one below:

```python
@classmethod
def __torch_function__(cls, func, types, args=(), kwargs=None):
    if kwargs is None:
        kwargs = {}
    if func not in HANDLED_FUNCTIONS or not all(
            issubclass(t, (torch.Tensor, ScalarTensor)) for t in types):
        args = [a.tensor() if hasattr(a, 'tensor') else a for a in args]
        return func(*args, **kwargs)
    return HANDLED_FUNCTIONS[func](*args, **kwargs)
```

In [20]:
HANDLED_FUNCTIONS = {}
class ScalarTensor(object):
    def __init__(self, N, value):
        self._N = N
        self._value = value

    def __repr__(self):
        return "DiagonalTensor(N={}, value={})".format(self._N, self._value)

    def tensor(self):
        return self._value * torch.eye(self._N)

    @classmethod
    def __torch_function__(cls, func, types, args=(), kwargs=None):
        if kwargs is None:
            kwargs = {}
        if func not in HANDLED_FUNCTIONS or not all(
                issubclass(t, (torch.Tensor, ScalarTensor))
                for t in types
            ):
            args = [a.tensor() if hasattr(a, 'tensor') else a for a in args]
            return func(*args, **kwargs)
        return HANDLED_FUNCTIONS[func](*args, **kwargs)

Then <a href="https://pytorch.org/docs/stable/generated/torch.mul.html#torch.mul" style="text-decoration:none;">torch.mul()</a> will work correctly, although the return type will always be a `Tensor` rather than a `ScalarTensor`, even if both operands are `ScalarTensor` instances:

In [21]:
s = ScalarTensor(2, 2)
torch.mul(s, s)

tensor([[4., 0.],
        [0., 4.]])

<br>

<font size=3>Also see the `MetadataTensor` example below for another variation on this pattern but instead always returns a `MetadataTensor` to propagate metadata through operations in the `torch` API.


The `__torch_function__` protocol is designed for full coverage of the API, partial coverage may lead to undesirable results, in particular, certain functions raising a `TypeError`. This is especially true for subclasses, where all three of ***torch.add***, ***torch.Tensor.__add__*** and ***torch.Tensor.add*** must be covered, even if they return exactly the same result. Failing to do this may also lead to infinite recursion. <br>
If one requires the implementation of a function from `torch.Tensor` subclasses, they must use `super().__torch_function__` inside their implementation.
</font>

<br>
<br>

## Subclassing <font color=maroon><b>torch.Tensor</font>

As of version 1.7.0, **`methods`**` on torch.Tensor` and **`functions`**` in public torch.*` namespaces applied on `torch.Tensor` subclasses will return subclass instances instead of `torch.Tensor` instances:

In [29]:
class SubTensor(torch.Tensor):
    pass

In [30]:
type(torch.add(SubTensor([0]), SubTensor([1]))).__name__

'SubTensor'

In [31]:
type(torch.add(SubTensor([0]), torch.tensor([1]))).__name__

'SubTensor'

<br>

If multiple subclasses exist, the lowest one in the hierarchy will be chosen by default. If there is no unique way to determine such a case, then a `TypeError` is raised:

In [34]:
class SubTensor2(SubTensor):
    pass

In [35]:
type(torch.add(SubTensor2([0]), SubTensor([1]))).__name__

'SubTensor2'

In [36]:
type(torch.add(SubTensor2([0]), torch.tensor([1]))).__name__

'SubTensor2'

<br>

In [40]:
class OtherSubTensor(torch.Tensor):
    pass

In [41]:
torch.add(SubTensor([0]), OtherSubTensor([1]))

TypeError: no implementation found for 'torch.add' on types that implement __torch_function__: [000001B58C138D50, 000001B58C0763F0]

原文档中的：
```python
TypeError: no implementation found for 'torch.add' on types that implement __torch_function__: [SubTensor, OtherSubTensor]
```

<br>

<font size=3>If one wishes to have a global override for all tensor methods, one can use `__torch_function__`. Here is an example that logs all function/method calls:</font>

```python
class LoggingTensor(torch.Tensor):
    @classmethod
    def __torch_function__(cls, func, types, args=(), kwargs=None):
        # NOTE: Logging calls Tensor.__repr__, so we can't log __repr__ without infinite recursion
        if func is not torch.Tensor.__repr__:
            logging.info(f"func: {func.__name__}, args: {args!r}, kwargs: {kwargs!r}")
        if kwargs is None:
            kwargs = {}
        return super().__torch_function__(func, types, args, kwargs)
```

<font size=3>However, if one instead wishes to override a method on the Tensor subclass, there one can do so either by directly overriding the method (by defining it for a subclass), or by using `__torch_function__` and matching with `func`.</font>

One should be careful within `__torch_function__` for subclasses to always call `super().__torch_function__(func, ...)` instead of func directly, as was the case before version 1.7.0. Failing to do this may cause func to recurse back into `__torch_function__` and therefore cause infinite recursion.

<br>
<br>

## Extending <font color=maroon>torch</font> with a <font color=maroon><b>Tensor wrapper</b></font> type

(暂略)

<br>
<br>

## Operations on multiple types that define <font color=maroon>__torch_function__</font>

(暂略)

<br>
<br>

## Testing Coverage of Overrides for the PyTorch API

(暂略)

<br>
<br>
<br>

# Writing custom C++ extensions

See this <a href="https://pytorch.org/tutorials/advanced/cpp_extension.html" style="text-decoration:none;"><font color=maroon>PyTorch tutorial</font></a> `Tutorials > Custom C++ and CUDA Extensions` for a detailed explanation and examples.

Documentations are available at <a href="https://pytorch.org/docs/stable/cpp_extension.html" style="text-decoration:none;"><font color=maroon>torch.utils.cpp_extension</font></a>.

<br>
<br>
<br>

# Writing custom C extensions

Example available at <a href="https://github.com/pytorch/extension-ffi" style="text-decoration:none;"><font color=maroon>this GitHub repository</font></a>.

<br>
<br>
<br>